## CRAWL dari REPUBLIKA

### Def untuk crawling dari Portal REPUBLIKA
Note : 
- ganti User-Agent pada headers -> ke user agent browser sendiri

In [21]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import os


def republika(keyword, n_berita):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
    }
    save_dir =  r"/home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/DATA/corpus/republika"

    data_republika = []
    page = 1

    while len(data_republika) < n_berita:
        url = f'https://republika.co.id/search/v3/?q={keyword}&sortby=time&page={page}'
        print(f"\n🔄 Scraping halaman {page}...")
        res = requests.get(url, headers=headers)

        if res.status_code != 200:
            print("❌ Gagal mengakses halaman.")
            break

        soup = BeautifulSoup(res.content, 'html.parser')
        articles = soup.find_all('div', class_='news-item')

        if not articles:
            print("❌ Tidak menemukan artikel lagi.")
            break

        for article in articles:
            try:
                link_tag = article.find('a', href=True)
                if not link_tag:
                    continue

                url_berita = link_tag['href']
                judul_tag = article.find('div', class_='news-title')
                judul = judul_tag.get_text(strip=True) if judul_tag else link_tag.get_text(strip=True)

                tanggal_tag = article.find('div', class_='news-source')
                tanggal = tanggal_tag.get_text(strip=True) if tanggal_tag else "?"

                detail_res = requests.get(url_berita, headers=headers)
                detail_soup = BeautifulSoup(detail_res.content, 'html.parser')
                konten = detail_soup.find('div', class_='main-content') or detail_soup.find('div', class_='body-text')

                isi = konten.get_text(separator=' ', strip=True) if konten else "Tidak ditemukan"

                if len(isi) > 30:
                    data_republika.append({
                        'judul': judul,
                        'berita': isi,
                        'url': url_berita,
                        'tanggal': tanggal
                    })
                    print(f"[{len(data_republika)}] {judul}")

                    if len(data_republika) >= n_berita:
                        break

                time.sleep(1)

            except Exception as e:
                print("⚠️ Error saat ambil artikel:", e)
                continue

        page += 1

    if data_republika:
            filename = os.path.join(save_dir, f"republika_news_{keyword.lower().replace(' ','_')}_{time.strftime('%Y%m%d')}.csv")
            with open(filename, "w", newline="", encoding="utf-8-sig") as f:
                writer = csv.DictWriter(f, fieldnames=["judul", "berita", "url"])
                writer.writeheader()
                writer.writerows(data_republika)
            print(f"\n✅ Selesai! {len(data_republika)} artikel '{keyword}' disimpan ke {filename}")
    else:
            print(f"❌ Tidak ada data yang ditemukan untuk '{keyword}'")

In [ ]:
keywords = [
    "BI rate",
    "Pertumbuhan ekonomi",
    "Inflasi",
    "Bank Indonesia",
    "Perbankan digital",
    "IHSG",
    "Pelemahan Rupiah",
    "Investasi",
    "Ekonomi Indonesia",
    "Danantara",
    "OJK",
    "Reksadana",
    "Bursa Efek Indonesia",
    "BEI",
    "Ekonomi digital"
]

republika_batch(keywords, n_berita=100)


In [22]:
republika(keyword="IHSG", n_berita=100)


🔄 Scraping halaman 1...
[1] IHSG Dibuka Anjlok, Tertekan Sentimen Global dan Penguatan Dolar AS
[2] Pasar Cermati Data Investasi Asing, IHSG Diprediksi Fluktuatif
[3] Pasar Modal Catat Rekor Baru, IHSG Ditutup di Level 8.257
[4] IHSG Cetak Rekor Tertinggi, Kapitalisasi Pasar Tembus Rp 14.995 Triliun
[5] Menkeu akan Beri Insentif BEI: IHSG akan Cenderung Naik Terus
[6] OJK-Kemenkeu Sepakat Perkuat Pasar Modal, Menkeu Janji Insentif Pajak jika Pasar Lebih Tertib
[7] Menkeu Purbaya Yakin IHSG Melesatto the Moon
[8] OJK Optimistis IHSG Tembus Level 8.000 pada Oktober
[9] IHSG Menguat 21 Poin, Didorong Sentimen Positif dari Asia
[10] IHSG Diprediksi Tembus 8.168, Sentimen Domestik dan Global Perkuat Arah Pasar
[11] Sepekan Terakhir, IHSG Ditutup di Level 8.118 dengan Kapitalisasi Pasar Rp15.079 Triliun
[12] IHSG Menguat 47 Poin di Akhir Pekan, Seiring Sentimen Positif Bursa Asia
[13] IHSG Dibuka Menguat ke 8.099, Pasar AbaikanShutdownAS
[14] IHSG Berpotensi Menguat di Tengah Shutdown Pemer

ValueError: dict contains fields not in fieldnames: 'tanggal'